# POP MOC(sigma 2) for 0.1-degree
**Input Data:** Monthly POP output timeseries files  
**Output Data:** Monthly mean AMOC sigma 2 timeseries  
**Description:** Computes MOC(sigma 2) offline from POP history files using simple xhistogram binning.  
**Date:** February 2023  
**Creator:** Steve Yeager (https://github.com/sgyeager/POP_MOC/blob/main/notebooks/pop_MOCsigma2_0.1deg.ipynb)  
**Updated:** Fred Castruccio and Teagan King, February 2023  
**Note:** To use the MOCutils, a user will need to clone the POP_MOC repository (https://github.com/sgyeager/POP_MOC) and install MOCutils by going to the POP_MOC directory and running `pip install -e . --user`.

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import glob
import dask
from xhistogram.xarray import histogram
import matplotlib.pyplot as plt
%matplotlib inline

from MOCutils import popmoc
import pop_tools

import os

In [2]:
from dask.distributed import wait
dask.__version__

'2022.7.0'

In [3]:
# Close out Dask Cluster and release workers:
#cluster.close()
#client.close()

In [6]:
# TODO: optimize dask resources

def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=10GB',
        account='P93300313',
        walltime='06:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(72) 

/glade/u/home/tking/.conda/envs/ipogs/lib/python3.10/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36828 instead
  warnings.warn(


In [26]:
cluster

AttributeError: 'Tab' object has no attribute '_ipython_display_'

PBSCluster(77a4225d, 'tcp://10.12.206.48:39709', workers=67, threads=67, memory=670.00 GiB)

In [8]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

# Get the required variables 

In [14]:
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002/ocn/proc/tseries/month_1/'
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003/ocn/proc/tseries/month_1/'
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP45/HR/b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP26/HR/b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'

#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.VVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.VVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
dsV = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsV = time_set_midmonth(dsV,'time')

#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.WVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.WVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
dsW = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsW = time_set_midmonth(dsW,'time')


fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.TEMP.200601-210012.nc'
dsT = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsT = time_set_midmonth(dsT,'time')

fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.SALT.200601-210012.nc'
dsS = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsS = time_set_midmonth(dsS,'time')

fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.UVEL.200601-210012.nc'
dsU = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsU = time_set_midmonth(dsU,'time')

# /glade/scratch/yeager/forTeagan is copy of Who Kim's data

fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)
#ds_grid = pop_tools.get_grid('/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc')

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

In [28]:
%%time
u_e_all = dsU['UVEL']
u_e_all = u_e_all.where(u_e_all<1.e30,0)
v_e_all = dsV['VVEL']
v_e_all = v_e_all.where(v_e_all<1.e30,0)

CPU times: user 9.24 ms, sys: 2.31 ms, total: 11.5 ms
Wall time: 16.7 ms


In [29]:
# Get model T & S
salt_all = dsS['SALT']
temp_all = dsT['TEMP']

In [30]:
tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

### MOC Region Mask

In [31]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [32]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

In [35]:
%%time
dz = ds_grid['dz'].persist() / 100.
print('got dz')
kmt = ds_grid['KMT'].fillna(0).persist() 
print('got kmt')
# Slow step (~12 mins)
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)
print('got dzt dzu')

got dz
got kmt
got dzt dzu
CPU times: user 1min 28s, sys: 3min 34s, total: 5min 2s
Wall time: 11min 10s


# Loop over time slices and compute MOC 

In [49]:
# ystart=[2006,2010,2020,2030,2040,2050,2060,2070,2080,2090,2100]
# yend=[2009,2019,2029,2039,2049,2059,2069,2079,2089,2099,2100]

ystart=[2006]
yend=[2006]

In [50]:
%%time
for n in range(len(ystart)):
    v_e = v_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    u_e = u_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    salt = salt_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    temp = temp_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))

    # Compute sigma-2 field from POP model output
    refz = 2000
    refdep = xr.full_like(dsV['z_t'],refz).rename('REFDEP')

    # Sigma2 on model TLAT, TLONG
    sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
    sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})
    
    print("refdep {}".format(refdep))
    print("sigma2_T {}".format(sigma2_T)) # TODO: Should have time, z_t, nlat, nlon dims
    
    # Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
    sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()
    print('sigma_mid'.format(sigma_mid))
    print('sigma_edge'.format(sigma_edge))

    # Compute Isopycnal Layer Thickness (Can skip if not needed)
    # Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
    iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
    iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

    kmtdiff = iso_count.sum('sigma') - ds_grid['KMT']
    print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))
    
    # Use histogram to compute layer thickness. Vertical sum should be same as HT.
    iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzt,dim=['z_t'],density=False)
    iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_thick = iso_thick.rename('iso_thick').assign_attrs({'units':'m',
                                                            'long_name':'Isopycnal Layer Thickness'}).rename({'sigma':
                                                                                                              'sigma_mid'})
    print('iso_thick'.format(iso_thick))
    # TODO: check time transpose with more than one time slice
    iso_thick = iso_thick.transpose('time','sigma_mid','nlat','nlon')
    
    # takes somewhat long time to run this cell... it's also not really using dask.
    htdiff = iso_thick.sum('sigma_mid') - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
    print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
    
    # Compute Isopycnal Layer Depth (Can skip if not needed)
    # Cumulative sum of layer thickness yields depth of layer edges:
    iso_depth = iso_thick.cumsum('sigma_mid').rename('iso_depth').rename({'sigma_mid':'sigma_bot'}).assign_attrs({'units':'m','long_name':'Isopycnal Layer Depth'})
    sigma_bot = sigma_edge.isel(sigma=slice(1,None)).rename({'sigma':'sigma_bot'}).assign_attrs({'long_name':'Sigma2 at bottom of layer'})
    iso_depth['sigma_bot'] = sigma_bot
    iso_depth = iso_depth.transpose('time', 'sigma_bot','nlat','nlon')
    print('iso_depth'.format(iso_depth))
    
    iso_depth.isel(time=0,sigma_bot=84).plot(size=6,vmax=5500)
    
    # Isopycnal depth of bottom-most layer should be same as HT.
    htdiff =  iso_depth.isel(sigma_bot=-1) - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
    print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
    
    # Compute Isopycnal Layer Horizontal Volume Flux
    # Grid Metrics
    print('computing grid metrics')
    dxu = ds_grid['DXU']
    dyu = ds_grid['DYU']
    dxt = ds_grid['DXT']
    dyt = ds_grid['DYT']
    
    # Grid-oriented Volume FLuxes:
    print('generating grid-oriented volume fluxes')
    u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    
    # Convert u_e,v_e to C-grid fluxes
    print('converting to c-grid fluxes')
    u = 0.5*(u_e+u_e.shift(nlat=1))
    v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))
    
    # Volume fluxes in density-space. 
    print('generating iso_flux')
    iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
    iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
    iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    
    print('performing dask operations')
    iso_uflux = iso_uflux.persist()
    daskout = wait(iso_uflux)
    iso_vflux = iso_vflux.persist()
    daskout = wait(iso_vflux)
    
    # Vertical sum in density-space should reproduce vertical sum in depth-space
    ufluxdiff = iso_uflux.isel(time=0).sum('sigma') - u.isel(time=0).sum('z_t')
    print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
    vfluxdiff = iso_vflux.isel(time=0).sum('sigma') - v.isel(time=0).sum('z_t')
    print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))
    
    # Compute Vertical Volume Flux from horizontal flux convergence
    print('computing vertical volume flux')
    wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
    wflux = wflux.assign_coords({'TLAT':ds['TLAT'],'TLONG':ds['TLONG']}).drop(['ULAT','ULONG'])
    wflux = wflux.persist()
    daskout = wait(wflux)
    
    # plot
    print('plotting')
    rmaskmoc.plot(levels=[0,1,2,3],col='transport_reg',size=5)
    
    # Compute MOC
    print('computing MOC')
    MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
    MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
    MOC = MOC.load()

    # add vflux at southern boundary of Atlantic domain
    tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
    tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
    moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
    moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
    MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s
    
    MOC.isel(time=0).isel(transport_reg=0).plot(ylim=[40,28])
    MOC.isel(time=0).isel(transport_reg=1).plot(ylim=[40,28])
    
    # Save to netcdf
    MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
    dsout = MOCann.to_dataset()
    
    outdir = os.path.dirname(fin)
    fout = os.path.split(fin)[-1].split('.')[:-3]
    fout.append('MOCsig')
    fout.append('{:04d}{:02d}-{:04d}{:02d}'.format(dsout.time.dt.year[0].values,dsout.time.dt.month[0].values,dsout.time.dt.year[-1].values,dsout.time.dt.month[-1].values))
    fout.append('nc')
    fout = '.'.join(fout)
    fout = os.path.join(outdir,fout)
    
    dso.to_netcdf(fout)
    dsout.to_netcdf(fout,unlimited_dims='time')

MemoryError: Unable to allocate 2.22 TiB for an array with shape (610675200000,) and data type float32

In [51]:
v_e = v_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
u_e = u_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
salt = salt_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
temp = temp_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))

# Compute sigma-2 field from POP model output
refz = 2000
refdep = xr.full_like(dsV['z_t'],refz).rename('REFDEP')

In [ ]:
# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

In [ ]:
print("refdep {}".format(refdep))
print("sigma2_T {}".format(sigma2_T)) # TODO: Should have time, z_t, nlat, nlon dims

# Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()
print('sigma_mid'.format(sigma_mid))
print('sigma_edge'.format(sigma_edge))

In [ ]:
# Compute Isopycnal Layer Thickness (Can skip if not needed)
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

kmtdiff = iso_count.sum('sigma') - ds_grid['KMT']
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

# Use histogram to compute layer thickness. Vertical sum should be same as HT.
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzt,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('iso_thick').assign_attrs({'units':'m',
                                                        'long_name':'Isopycnal Layer Thickness'}).rename({'sigma':
                                                                                                          'sigma_mid'})
print('iso_thick'.format(iso_thick))
# TODO: check time transpose with more than one time slice
iso_thick = iso_thick.transpose('time','sigma_mid','nlat','nlon')

# takes somewhat long time to run this cell... it's also not really using dask.
htdiff = iso_thick.sum('sigma_mid') - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

# Compute Isopycnal Layer Depth (Can skip if not needed)
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma_mid').rename('iso_depth').rename({'sigma_mid':'sigma_bot'}).assign_attrs({'units':'m','long_name':'Isopycnal Layer Depth'})
sigma_bot = sigma_edge.isel(sigma=slice(1,None)).rename({'sigma':'sigma_bot'}).assign_attrs({'long_name':'Sigma2 at bottom of layer'})
iso_depth['sigma_bot'] = sigma_bot
iso_depth = iso_depth.transpose('time', 'sigma_bot','nlat','nlon')
print('iso_depth'.format(iso_depth))

iso_depth.isel(time=0,sigma_bot=84).plot(size=6,vmax=5500)

# Isopycnal depth of bottom-most layer should be same as HT.
htdiff =  iso_depth.isel(sigma_bot=-1) - (ds_grid['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

# Compute Isopycnal Layer Horizontal Volume Flux
# Grid Metrics
print('computing grid metrics')
dxu = ds_grid['DXU']
dyu = ds_grid['DYU']
dxt = ds_grid['DXT']
dyt = ds_grid['DYT']

# Grid-oriented Volume FLuxes:
print('generating grid-oriented volume fluxes')
u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})

# Convert u_e,v_e to C-grid fluxes
print('converting to c-grid fluxes')
u = 0.5*(u_e+u_e.shift(nlat=1))
v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

# Volume fluxes in density-space. 
print('generating iso_flux')
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

print('performing dask operations')
iso_uflux = iso_uflux.persist()
daskout = wait(iso_uflux)
iso_vflux = iso_vflux.persist()
daskout = wait(iso_vflux)

# Vertical sum in density-space should reproduce vertical sum in depth-space
ufluxdiff = iso_uflux.isel(time=0).sum('sigma') - u.isel(time=0).sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
vfluxdiff = iso_vflux.isel(time=0).sum('sigma') - v.isel(time=0).sum('z_t')
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))

# Compute Vertical Volume Flux from horizontal flux convergence
print('computing vertical volume flux')
wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
wflux = wflux.assign_coords({'TLAT':ds['TLAT'],'TLONG':ds['TLONG']}).drop(['ULAT','ULONG'])
wflux = wflux.persist()
daskout = wait(wflux)

# plot
print('plotting')
rmaskmoc.plot(levels=[0,1,2,3],col='transport_reg',size=5)

# Compute MOC
print('computing MOC')
MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
MOC = MOC.load()

# add vflux at southern boundary of Atlantic domain
tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

MOC.isel(time=0).isel(transport_reg=0).plot(ylim=[40,28])
MOC.isel(time=0).isel(transport_reg=1).plot(ylim=[40,28])

# Save to netcdf
MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
dsout = MOCann.to_dataset()

outdir = os.path.dirname(fin)
fout = os.path.split(fin)[-1].split('.')[:-3]
fout.append('MOCsig')
fout.append('{:04d}{:02d}-{:04d}{:02d}'.format(dsout.time.dt.year[0].values,dsout.time.dt.month[0].values,dsout.time.dt.year[-1].values,dsout.time.dt.month[-1].values))
fout.append('nc')
fout = '.'.join(fout)
fout = os.path.join(outdir,fout)

dso.to_netcdf(fout)
dsout.to_netcdf(fout,unlimited_dims='time')